# RDDs - wprowadzenie

RDD czyli Resilient Distributed Dataset. RDD API zostało wydane w 2011 r, co oznacza, że od powstania Sparka to RDD API było głównym interfejsem, z którego mogli korzystać użytkownicy Sparka. API to Application Programmers Interface i jest to w zasadzie zestaw bibliotek, które są udostępniane w celu rozszerzenia funkcjonalności lub możliwości określonego języka programowania. Na dzień dzisiejszy możemy RDD traktować jako API (jeśli decydujemy się na korzystanie z ramek pysparkowych) albo jak zbiór rozproszonych danych. 

RDD, jak nazwa wskazuje, jest odpornym, rozproszonym, niezmiennym (immutable) zbiorem danych. Dane są podzielone na węzły w klastrze. RDD jest zawsze definiowany z alokacją danych, a akcje i transformacje w żaden sposób nie zmieniają jego kształtu. Dowolna z nich zawsze zwróci nowy RDD, przechowujący wyniki operacji. Dlatego działania wykonywane na RDD mogą się dziać równolegle. RDD muszą być odporne, ponieważ klastry mogą ulec awarii. RDD to interfejsy API niskiego poziomu, które nie jest łatwo zrozumieć jeżeli do tej pory korzystało się głównie z ustrukturyzowanych interfejsów API wysokiego poziomu, takich jak ramki danych i zbiory danych. Niemniej RDDsy trzeba znać i rozumieć żeby móc sprawnie przekształcać nieustrukturyzowane dane strumieniowe, których nie sposób okiełznać używając wysokopoziomiowych ramek. Innym powodem jest konieczność dogłębnego zrozumienia działania Sparka na wypadek potrzeby zajrzenia pod mechanizmy, które wykorzystują ramki. Zrozumienie działania RDD bardzo pomoże w optymalizacji działania aplikacji Sparkowych. Od czasów Spark 2 (i wprowadzenia wysokopoziomowych API) RDD traktowane są jak relikt przeszłości, ale jest to zgubne podejście. Dlatego powinniśmy się nad nimi pochylić.

Mamy za sobą wstęp teoretyczny, pora na kilka linijek praktyki. Wykonamy na RDD kilka prostych operacji korzystając przy tym z podstawowych funkcji. Musimy pamiętać, że pracując na RDDsach pracujemy na obiektach Javowych, które różnią się od ramek sparkowych. Przygotujemy teraz obiekt do modelowania: 


In [3]:

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TestRDD").getOrCreate()


slowa = "Spark znacznie ułatwia życie i wprawia mnie w dobry nastrój, Spark jest fantastyczny".split(" ")
type(slowa)

list

In [4]:
print(slowa) #faktycznie lista

['Spark', 'znacznie', 'ułatwia', 'życie', 'i', 'wprawia', 'mnie', 'w', 'dobry', 'nastrój,', 'Spark', 'jest', 'fantastyczny']


Teraz musimy stworzyć RDD, które tę listę słów będzie przechowywać.

In [5]:
slowa_rdd = spark.sparkContext.parallelize(slowa)
type(slowa_rdd)

pyspark.rdd.RDD

In [7]:
slowa_dane = slowa_rdd.collect() # sprawdzmy czy na pewno wszystko ok
# nie da się przepętlić przez RDD - trzeba je zebrać do nowego zbioru
for slowo in slowa_dane: 
    print(slowo)

Spark
znacznie
ułatwia
życie
i
wprawia
mnie
w
dobry
nastrój,
Spark
jest
fantastyczny


### Transformacja distinct 

Na RDDsach można wykonywać wiele różnych transformacji, my przyjrzymy się tylko kilku podstawowym. Zaczniemy od distincta, który pomoże nam pozbyć się duplikatów. 

Najpierw trzeba policzyć elementy w RDD: 

In [8]:
slowa_rdd.count()

13

In [9]:
slowa_rdd.distinct().count()

12

Policzyliśmy wszystkie słowa, policzyliśmy unikalne słowa. Ale czy je usunęliśmy?

In [12]:
slowa_dane = slowa_rdd.collect() #sprawdzmy czy na pewno wszystko ok
for slowo in slowa_dane: 
    print(slowo)

Spark
znacznie
ułatwia
życie
i
wprawia
mnie
w
dobry
nastrój,
Spark
jest
fantastyczny


Duplikaty nadal są, co potwierdza tylko, że dowolna transformacja RDD zwraca nowy zbiór danych, nie ingerując w zawartość zbioru pierwotnego. Żeby móc się realnie pozbyć duplikatów wynik transformacji trzeba zapisać do nowego zbioru

In [14]:
slowa_unique_rdd = slowa_rdd.distinct()
slowa_unique_dane = slowa_unique_rdd.collect()

for slowo in slowa_unique_dane: 
    print(slowo)

znacznie
wprawia
ułatwia
dobry
i
jest
mnie
Spark
życie
w
fantastyczny
nastrój,


Dubli brak. Teraz przefiltrujemy RDD, wybierzemy tylko te słowa, które zaczynają się na literę s:

In [18]:
# musimy napisać pod to własną funkcję:

def slowa_s(slowo,litera): 
    return slowo.startswith(litera) # skorzystamy z startswith() dotępne w Pythonie

slowa_unique_rdd.filter(lambda slowo: slowa_s(slowo,"S")).collect() #filtrując rekordy RDDs najczęściej trzeba będzie to robić korzystając jednocześnie z pythonowych funkcji anonimowych (lambda)

['Spark']

### Transformacje map i flatMap

Transformacja *map()* służy do wykonywania złożonych operacji, takich jak dodawanie kolumny lub zmiany/przekształcenia wartości kolumny. Dane wyjściowe po ww transformacji zawsze będą miały taką samą liczbę rekodów jak dane wejściowe. Przejdżmy do przykładu.

In [20]:
# zaczynamy od stworzenia listy numerów: 

num_list = [*range(1,21)]
print(num_list)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [24]:
nums_rdd = spark.sparkContext.parallelize(num_list) # tworzymy listę numerów
nums_squared_rdd = nums_rdd.map(lambda n: (n, n**2)) # transformujemy ją w listę tupli, które zawierają numer i jego kwadrat
for num in nums_squared_rdd.collect(): 
    print(num)

(1, 1)
(2, 4)
(3, 9)
(4, 16)
(5, 25)
(6, 36)
(7, 49)
(8, 64)
(9, 81)
(10, 100)
(11, 121)
(12, 144)
(13, 169)
(14, 196)
(15, 225)
(16, 256)
(17, 289)
(18, 324)
(19, 361)
(20, 400)


Podsumowując - jeśli chcemy przetransformować rdd, możemy to wykonać korzystając z funckji *map()*. Jeszcze jeden przykład:

In [27]:
slowa_trd_rdd = slowa_rdd.map(lambda word: (word, word[0], slowa_s(word,"S")))

for trd in slowa_trd_rdd.collect(): 
    print(trd)

('Spark', 'S', True)
('znacznie', 'z', False)
('ułatwia', 'u', False)
('życie', 'ż', False)
('i', 'i', False)
('wprawia', 'w', False)
('mnie', 'm', False)
('w', 'w', False)
('dobry', 'd', False)
('nastrój,', 'n', False)
('Spark', 'S', True)
('jest', 'j', False)
('fantastyczny', 'f', False)


Transformacja *flatMap()* jest rozszerzeniem transformacji *map()*. Możemy spotkać się z sytuacją, kiedy danymi wyjściowymi bedzie np. lista słów, a my z tej listy wyrazów będziemy musieli zrobić jedną, wielką, listę litter. Czyli potrzebujemy zejść poziom niżej niż to, co daje nam *map()*. Rzućmy okiem na poniższy przykład:

In [29]:
slowa_rdd.flatMap(lambda word: list(word)).take(10)

['S', 'p', 'a', 'r', 'k', 'z', 'n', 'a', 'c', 'z']

### Sortowanie przy użyciu SortBykey()

Ostatnia z przykładowych trnsformacji RDDs - sortowanie po kluczu. Na początek przygotujmy dane. Transformacja *SortByKey()* jako input przyjmuje parę klucz - wartość, więc żeby móc wykonać przykład, musimy stworzyć sobie tuplę. 

In [31]:
countries_list = [("India",91), ("USA",4), ("Greece",13)]
countries_rdd = spark.sparkContext.parallelize(countries_list)
srt_countries_list = countries_rdd.sortByKey().collect()

for country in srt_countries_list:
    print(country) 


('Greece', 13)
('India', 91)
('USA', 4)


In [35]:
srt_countries_list = countries_rdd.map(lambda c: (c[1], c[0])).sortByKey(False).collect()
# map() + lambda zamienia miejscami kolumny w liście żeby móc zwrócić coś, co ma sens
# false w sortByKey żeby zwrócić wartości od największej do najmniejszej

for country in srt_countries_list:
    print(country) 


(91, 'India')
(13, 'Greece')
(4, 'USA')
